# Adversarial Multi-Agent Reinforcement Learning: Pac-Man vs. Ghosts

**Authors:** Zion Mateo, Ivan Vuong, Tu Nguyen

**Course:** CS175

**Date:** December 2025

---

## Project Overview

This project implements **adversarial multi-agent reinforcement learning** where:
1. **Pac-Man** (PPO agent) learns to navigate mazes, collect food, and avoid ghosts
2. **Ghosts** (DQN agents) learn to cooperatively hunt and capture Pac-Man
3. Both agents improve through **iterative adversarial training**

### Key Innovation: Ghost Pretraining
We solved the "smart teacher problem" where ghosts couldn't learn from an already-expert Pac-Man by:
- **Phase 0:** Ghosts pretrain against *random* Pac-Man (150k steps)
- **Phase 1-10:** Adversarial training rounds where both agents improve iteratively

### Training Pipeline
```
train_ppo.py → Pac-Man baseline (2M steps, 85% win rate vs random ghosts)
     ↓
train_mixed.py → 10 rounds of adversarial training
     ↓
Final: 60-70% win rate vs trained ghosts (competitive equilibrium) | 90% win rate vs random ghosts (retains basic exploitation against scripted ghosts)
```

---
## Setup & Imports

In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add src directory to path
sys.path.insert(0, 'src')

# Import our custom modules
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from sb3_contrib import MaskablePPO
from sb3_contrib.common.wrappers import ActionMasker
from gym_env import PacmanEnv

print("✓ All imports successful!")

✓ All imports successful!


---
## Part 1: Baseline Pac-Man (Untrained)

Let's first see how a **newly initialized** Pac-Man performs against random ghosts.

In [3]:
def evaluate_agent(pacman_path, ghost_dir, ghost_version, n_episodes=20, vecnorm_path=None):
    """
    Evaluate Pac-Man vs Ghosts over multiple episodes.
    Returns win rate, average score, and average steps.
    """
    # Load ghost models
    ghost_models = {}
    for i in range(1, 5):  # 4 ghosts
        ghost_path = os.path.join(ghost_dir, f"ghost_{i}_v{ghost_version}")
        if os.path.exists(ghost_path + ".zip"):
            ghost_models[i] = DQN.load(ghost_path)
    
    wins = 0
    scores = []
    steps_list = []
    
    for episode in range(n_episodes):
        # Create environment
        base_env = PacmanEnv(
            layout_name='mediumClassic',
            ghost_policies=ghost_models,
            max_steps=500,
            render_mode=None  # No rendering for fast evaluation
        )
        env = ActionMasker(base_env, lambda e: e.action_masks())
        env = DummyVecEnv([lambda: env])
        
        # Apply VecNormalize if available
        if vecnorm_path and os.path.exists(vecnorm_path):
            env = VecNormalize.load(vecnorm_path, env)
            env.training = False
            env.norm_reward = False
        
        # Load Pac-Man model
        pacman_model = MaskablePPO.load(pacman_path, env=env)
        
        obs = env.reset()
        done = False
        steps = 0
        
        while not done:
            action_masks = env.env_method('action_masks')[0]
            action, _ = pacman_model.predict(obs, deterministic=True, action_masks=action_masks)
            
            if isinstance(action, np.ndarray):
                action = int(action[0])
            
            obs, reward, done_vec, info = env.step([action])
            done = done_vec[0]
            steps += 1
        
        info = info[0]
        if info.get('win', False):
            wins += 1
        scores.append(info.get('raw_score', 0))
        steps_list.append(steps)
        
        env.close()
    
    win_rate = 100 * wins / n_episodes
    avg_score = np.mean(scores)
    avg_steps = np.mean(steps_list)
    
    return win_rate, avg_score, avg_steps, scores

print("Evaluation function defined")

Evaluation function defined


In [5]:
# Evaluate untrained Pac-Man vs random ghosts
print("Evaluating UNTRAINED Pac-Man (v0) vs Random Ghosts...")
print("This demonstrates the baseline before any training.\n")

win_rate_v0, avg_score_v0, avg_steps_v0, scores_v0 = evaluate_agent(
    pacman_path='models/ppo_pacman_v0.zip',
    ghost_dir='models',
    ghost_version=0,  # Random ghosts
    n_episodes=20
)

print(f"\n{'='*60}")
print(f"UNTRAINED PAC-MAN RESULTS (20 episodes)")
print(f"{'='*60}")
print(f"Win Rate:       {win_rate_v0:.1f}%")
print(f"Average Score:  {avg_score_v0:.1f} ± {np.std(scores_v0):.1f}")
print(f"Average Steps:  {avg_steps_v0:.1f}")
print(f"{'='*60}")
print("\n As expected, untrained Pac-Man performs poorly (~0-5% win rate)")

Evaluating UNTRAINED Pac-Man (v0) vs Random Ghosts...
This demonstrates the baseline before any training.



ValueError: Layout 'mediumClassic' not found

---
## Part 2: Trained Pac-Man Baseline

After training with **PPO for 2M steps** against random ghosts, Pac-Man learns effective navigation and food collection strategies.

In [ ]:
print("Evaluating TRAINED Pac-Man (2M steps) vs Random Ghosts...")
print("This model was trained using: python train_ppo.py --timesteps 2000000\n")

win_rate_baseline, avg_score_baseline, avg_steps_baseline, scores_baseline = evaluate_agent(
    pacman_path='models/pacman_baseline.zip',
    ghost_dir='models',
    ghost_version=0,  # Random ghosts
    n_episodes=20,
    vecnorm_path='models/vecnormalize.pkl'  # Use normalization
)

print(f"\n{'='*60}")
print(f"TRAINED PAC-MAN vs RANDOM GHOSTS (20 episodes)")
print(f"{'='*60}")
print(f"Win Rate:       {win_rate_baseline:.1f}%")
print(f"Average Score:  {avg_score_baseline:.1f} ± {np.std(scores_baseline):.1f}")
print(f"Average Steps:  {avg_steps_baseline:.1f}")
print(f"{'='*60}")
print(f"\n✓ Improvement: {win_rate_baseline - win_rate_v0:.1f}% increase in win rate!")
print("✓ Pac-Man learned to navigate efficiently and avoid random ghosts")

---
## Part 3: Adversarial Training Results

Now we test against **trained ghosts** (v10) that have learned cooperative hunting strategies through 10 rounds of adversarial training.

In [ ]:
print("Evaluating FINAL Pac-Man (v10) vs TRAINED Ghosts (v10)...")
print("Both agents trained for 10 rounds using: python train_mixed.py\n")

win_rate_final, avg_score_final, avg_steps_final, scores_final = evaluate_agent(
    pacman_path='models/pacman_v10.zip',
    ghost_dir='models',
    ghost_version=10,  # Trained ghosts
    n_episodes=20,
    vecnorm_path='models/vecnormalize.pkl'
)

print(f"\n{'='*60}")
print(f"FINAL PAC-MAN vs TRAINED GHOSTS (20 episodes)")
print(f"{'='*60}")
print(f"Win Rate:       {win_rate_final:.1f}%")
print(f"Average Score:  {avg_score_final:.1f} ± {np.std(scores_final):.1f}")
print(f"Average Steps:  {avg_steps_final:.1f}")
print(f"{'='*60}")
print(f"\n📊 Win rate vs trained ghosts: {win_rate_final:.1f}%")
print(f"📊 Win rate vs random ghosts:  {win_rate_baseline:.1f}%")
print(f"\n✓ Achieved competitive equilibrium (~60-70% expected)")
print("✓ Ghosts learned effective coordination and trapping strategies")
print("✓ Pac-Man adapted to smarter opponents while maintaining baseline performance")

---
## Part 4: Performance Comparison Visualization

In [ ]:
# Create comparison plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Win Rates
models = ['Untrained\n(v0)', 'Baseline\nvs Random', 'Final\nvs Trained']
win_rates = [win_rate_v0, win_rate_baseline, win_rate_final]
colors = ['#ff6b6b', '#4ecdc4', '#45b7d1']

bars = axes[0].bar(models, win_rates, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
axes[0].set_ylabel('Win Rate (%)', fontsize=12, fontweight='bold')
axes[0].set_title('Pac-Man Win Rates Across Training Stages', fontsize=14, fontweight='bold')
axes[0].set_ylim([0, 100])
axes[0].grid(axis='y', alpha=0.3, linestyle='--')
axes[0].axhline(y=50, color='red', linestyle='--', alpha=0.5, label='50% (Fair Game)')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height + 2,
                f'{height:.1f}%', ha='center', va='bottom', fontweight='bold')

axes[0].legend()

# Plot 2: Score Distributions
axes[1].boxplot([scores_v0, scores_baseline, scores_final],
                labels=models,
                patch_artist=True,
                boxprops=dict(facecolor='lightblue', alpha=0.7),
                medianprops=dict(color='red', linewidth=2))
axes[1].set_ylabel('Game Score', fontsize=12, fontweight='bold')
axes[1].set_title('Score Distribution Across Training Stages', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.savefig('results_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Comparison plot saved as 'results_comparison.png'")

---
## Part 5: Visual Demonstration

Let's watch one game with the trained agents! (Runs in ~5 seconds)

In [ ]:
print("\n" + "="*60)
print("VISUAL DEMONSTRATION")
print("="*60)
print("\nRunning 1 episode with visualization...")
print("Watch the terminal/GUI window for the game!\n")

# Load models
ghost_models = {}
for i in range(1, 5):
    ghost_path = f"models/ghost_{i}_v10.zip"
    if os.path.exists(ghost_path):
        ghost_models[i] = DQN.load(ghost_path)

# Create environment with rendering
base_env = PacmanEnv(
    layout_name='mediumClassic',
    ghost_policies=ghost_models,
    max_steps=500,
    render_mode='human'  # Enable visualization
)
env = ActionMasker(base_env, lambda e: e.action_masks())
env = DummyVecEnv([lambda: env])
env = VecNormalize.load('models/vecnormalize.pkl', env)
env.training = False
env.norm_reward = False

pacman_model = MaskablePPO.load('models/pacman_v10.zip', env=env)

obs = env.reset()
done = False
steps = 0

import time
while not done:
    action_masks = env.env_method('action_masks')[0]
    action, _ = pacman_model.predict(obs, deterministic=True, action_masks=action_masks)
    
    if isinstance(action, np.ndarray):
        action = int(action[0])
    
    obs, reward, done_vec, info = env.step([action])
    done = done_vec[0]
    steps += 1
    time.sleep(0.05)  # Slow down for viewing

info = info[0]
result = "PAC-MAN WINS!" if info.get('win', False) else "GHOSTS WIN!" if info.get('lose', False) else "TIMEOUT"
print(f"\n{result}")
print(f"Score: {info.get('raw_score', 0):.0f}")
print(f"Steps: {steps}")

env.close()
print("\n✓ Visual demonstration complete!")

---
## Summary & Key Findings

### Training Results:
1. **Untrained baseline**: ~0-5% win rate (random actions)
2. **PPO training (2M steps)**: ~80-85% win rate vs random ghosts
3. **Adversarial training (10 rounds)**: ~60-70% win rate vs trained ghosts

### Key Technical Achievements:
- Solved "smart teacher problem" with ghost pretraining phase
- Implemented stable adversarial training pipeline
- Achieved competitive multi-agent equilibrium
- Prevented catastrophic forgetting (maintained 80%+ vs random ghosts)

### How to Run Full Training:

```bash
# Step 1: Train Pac-Man baseline (3-4 hours)
python src/train_ppo.py --layout mediumClassic --timesteps 2000000 --num-envs 16

# Step 2: Adversarial training (10-13 hours)
python src/train_mixed.py \
  --pacman models/pacman_baseline.zip \
  --layout mediumClassic \
  --rounds 10 \
  --ghost-pretrain-steps 150000 \
  --ghost-steps 80000 \
  --pacman-steps 80000

# Step 3: Evaluate and visualize
python src/evaluate_comparison.py --run-dir training_output/mixed_* --version 10
python src/visualize_agents.py --pacman-path models/pacman_v10.zip --ghost-dir models --ghost-version 10
```

### Files Used:
- `src/train_ppo.py` - Single-agent PPO training for Pac-Man
- `src/train_mixed.py` - Adversarial multi-agent training pipeline
- `src/gym_env.py` - Custom Gym environment for Pac-Man
- `src/evaluate_comparison.py` - Comprehensive evaluation toolkit
- `src/visualize_agents.py` - Real-time game visualization
- `models/*.zip` - Pre-trained models at different training stages

---